In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# 1. 读取 STL 文件
stl_file = "/home/lgx/Project/AFP/src/afp_mjc/env/flange_model2/meshes/flange_model.STL"
mesh = o3d.io.read_triangle_mesh(stl_file)

# 检查是否读取成功
if not mesh.has_triangles():
    print("Failed to read the STL file or the mesh has no triangles.")
    exit()

print(f"原始网格包含 {len(mesh.vertices)} 个顶点和 {len(mesh.triangles)} 个面")

原始网格包含 1881 个顶点和 978 个面


In [3]:
# 2. 采样生成点云
# number_of_points: 你希望生成的点数量（例如 10000 个点）
# init_factor: 初始采样倍数，值越大采样越均匀，但耗时越长，默认5即可
pcd = mesh.sample_points_poisson_disk(number_of_points=10000, init_factor=5)

# 如果泊松采样报错（极少数破损严重的网格），可以使用普通均匀采样代替：
# pcd = mesh.sample_points_uniformly(number_of_points=10000)

print(f"生成了 {len(pcd.points)} 个点")

生成了 10000 个点


In [4]:
# 3. 可视化对比
# 给原来的 mesh 计算法线以便显示更好看
mesh.compute_vertex_normals()

# 创建可视化窗口，左边是Mesh，右边是点云
pcd.translate((0.0, 0, 0)) # 把点云往右挪一点，防止重叠
# ================= 关键修改在这里 =================
# 创建坐标轴
# size: 轴的长度（根据你的模型尺寸调整，比如模型是毫米单位，可能需要设大点，如 100）
# origin: 坐标轴的原点位置，默认是 [0,0,0]
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# 如果你也想给那个被平移出去的点云单独加个坐标轴，可以再创建一个：
axes_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
# ================================================

# 在列表中把 axes 加进去
# o3d.visualization.draw_geometries([mesh, pcd, axes, axes_pcd], window_name="STL vs PointCloud")
o3d.visualization.draw_geometries([pcd, axes_pcd], window_name="STL vs PointCloud")

In [5]:
import yaml
import numpy as np

# 2.1 读取变换矩阵
yaml_path = '/home/lgx/Project/AFP/src/il_capture/config/gravity_tare.yaml'

try:
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    
    # 读取矩阵 (注意：yaml中你的键名是 T_frange_to_mocap)
    T_robot_to_mocap = np.array(data['T_robot_to_mocap'])
    T_flange_to_mocap = np.array(data['T_frange_to_mocap'])
    
    print("成功加载变换矩阵。")
    
except FileNotFoundError:
    print(f"Error: 找不到文件 {yaml_path}")
    # 这里为了演示，如果没有文件，给出单位矩阵，防止报错
    T_robot_to_mocap = np.eye(4)
    T_flange_to_mocap = np.eye(4)

# 2.2 计算目标变换矩阵 (Flange -> Robot)
# 公式: T_target = inv(T_robot_to_mocap) * T_flange_to_mocap
T_robot_to_mocap_inv = np.linalg.inv(T_robot_to_mocap)
T_target = T_robot_to_mocap_inv @ T_flange_to_mocap

print("计算出的变换矩阵 (Flange -> Robot):")
print(T_target)

pcd.transform(T_target)

print("点云坐标变换完成 (已转换到机械臂基座坐标系)")

成功加载变换矩阵。
计算出的变换矩阵 (Flange -> Robot):
[[ 0.00622606 -0.0019686   0.99997851 -0.65603855]
 [ 0.99998055 -0.00110112 -0.00622818 -0.45443915]
 [ 0.00111368  0.99999746  0.00196117 -0.01652494]
 [ 0.          0.          0.          1.        ]]
点云坐标变换完成 (已转换到机械臂基座坐标系)


In [6]:
# 4. 保存点云文件 (可选)
# 可以保存为 .pcd 或 .ply 格式
import os

data_directory = '/home/lgx/Project/AFP/src/il_capture/data/pointcloud'

os.makedirs(data_directory, exist_ok=True)

pcd_save_path = os.path.join(data_directory, "total_surface.pcd")
print(f"Saving point cloud to: {pcd_save_path}")

# 检查写入是否成功，并给出反馈
if o3d.io.write_point_cloud(pcd_save_path, pcd):
    print("Point cloud saved successfully.")
else:
    # 如果仍然失败，这可能是权限问题
    print("Failed to save the point cloud. Please check directory permissions.")


Saving point cloud to: /home/lgx/Project/AFP/src/il_capture/data/pointcloud/total_surface.pcd
Point cloud saved successfully.


# 发布到ros

In [7]:
import rospy
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
from std_msgs.msg import Header

points_np = np.asarray(pcd.points)

rospy.init_node('pointcloud_publisher', anonymous=True)
pub = rospy.Publisher('/flange_pointcloud', PointCloud2, queue_size=10)

header = Header()
header.frame_id = "world"
header.stamp = rospy.Time.now()

cloud_msg = pc2.create_cloud_xyz32(header, points_np.tolist())

rate = rospy.Rate(50)  # 50 Hz



while not rospy.is_shutdown():
    pub.publish(cloud_msg)
    rate.sleep()